Import necessary libraries: using Tensorflow framework to train the model

In [ ]:
from PIL import Image
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout


In [ ]:
#import the data set from my drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Data Preprocessing: 
In this section the following actions were executed. 

1.   Rename each class of image: rename the images using the three given classes: Bed, Sofa, Chair. 
2.   create a new folder

1.   Resize the images: resize the images for fine tuned model parameters
2.   List item





In [ ]:

folder_path = '/content/drive/MyDrive/NneamakaProjects/Data for test/Sofa'

prefix = 'Sofa_'
count = 1
for filename in os.listdir(folder_path):
  if filename.endswith('.jpg'):
        #old_name = os.path.join(folder_path, filename)
        new_name = prefix + str(count) + '.jpg'
        count += 1
        os.rename(os.path.join(folder_path, filename), os.path.join(folder_path, new_name))

    

In [ ]:
folder_path = '/content/drive/MyDrive/NneamakaProjects/Data for test/Chair'

prefix = 'Chair_'
count = 1
for filename in os.listdir(folder_path):
  if filename.endswith('.jpg'):
        #old_name = os.path.join(folder_path, filename)
        new_name = prefix + str(count) + '.jpg'
        count += 1
        os.rename(os.path.join(folder_path, filename), os.path.join(folder_path, new_name))

    


In [ ]:

folder_path = '/content/drive/MyDrive/NneamakaProjects/Data for test/Bed'

prefix = 'Bed_'
count = 1
for filename in os.listdir(folder_path):
  if filename.endswith('.jpg'):
        #old_name = os.path.join(folder_path, filename)
        new_name = prefix + str(count) + '.jpg'
        count += 1
        os.rename(os.path.join(folder_path, filename), os.path.join(folder_path, new_name))

    


In [ ]:
def resize_images(input_dir, output_dir, size):
    """
    Resizes all images in input_dir to size and saves them in output_dir.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Loop over all files in the input directory
    for filename in os.listdir(input_dir):
        # Only process image files
        if filename.endswith('.jpg') or filename.endswith('.png'):
            # Load the image
            image = Image.open(os.path.join(input_dir, filename))
            # Resize the image
            resized_image = image.resize(size)
            # Save the resized image to the output directory
            resized_image.save(os.path.join(output_dir, filename))


In [ ]:
input_dir = '/content/drive/MyDrive/NneamakaProjects/Train'
output_dir = '/content/drive/MyDrive/NneamakaProjects/train_r'
size = (256, 256)  # target size of the images

resize_images(input_dir, output_dir, size)


In [ ]:
input_dir = '/content/drive/MyDrive/NneamakaProjects/test'
output_dir = '/content/drive/MyDrive/NneamakaProjects/test_r'
size = (256, 256)  # target size of the images

resize_images(input_dir, output_dir, size)

# Model Training:
This section includes the following actions:

1.   Generate the model
2.   Train the model using fit_generator method

1.   Evaluate method using evaluate_generator method





In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/NneamakaProjects/train_r',
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/NneamakaProjects/test_r',
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical')


Found 240 images belonging to 3 classes.
Found 60 images belonging to 3 classes.


In [80]:
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(lr=1e-4),
              metrics=['accuracy'])
model.save('/content/drive/MyDrive/NneamakaProjects')

In [ ]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=10,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=10)


<ipython-input-79-c13f041cecc9>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/10
 8/10 [=======================>......] - ETA: 11s - loss: 0.7865 - accuracy: 0.6750

10/10 [==============================] - 52s 5s/step - loss: 0.7865 - accuracy: 0.6750 - val_loss: 0.5864 - val_accuracy: 0.7833


In [ ]:
test_loss, test_acc = model.evaluate_generator(validation_generator, steps=15)
print('test acc:', test_acc)


<ipython-input-78-9764bdf5222c>:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  test_loss, test_acc = model.evaluate_generator(validation_generator, steps=15)


test acc: 0.699999988079071


# Build an API:

The kind of API implemented below is the FAST API. It functions thus:

1.   Receives an input item- image
2.   predicts a label based on the class of label the model was train on. It should retuen an output label of either - Sofa, Chair, or Bed.



In [88]:
!pip install uvicorn
!pip install FastAPI
!pip install python-multipart


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for python-multipart: filename=python_multipart-0.0.5-py3-none-any.whl size=31678 sha256=c1e7c6933f660628e3121cf7da0d97827cfcabf17329fda25edb207fda36c495
  Stored in directory: /root/.cache/pip/wheels/9e/fc/1c/cf980e6413d3ee8e70cd8f39e2366b0f487e3e221aeb452eb0
Successfully built python-multipart


In [93]:
import uvicorn
from fastapi import FastAPI, File, UploadFile
from PIL import Image
from io import BytesIO
import tensorflow as tf
import numpy as np

# Load the pre-trained image classification model
model = tf.keras.models.load_model('/content/drive/MyDrive/NneamakaProjects')

# Define the FastAPI app
app = FastAPI()

# Define the endpoint for image classification
@app.post("/predict")
async def predict(image: UploadFile = File(...)):
    # Load the image file
    data = await image.read()
    img = Image.open(BytesIO(data))
    
    # Preprocess the image
    img = img.resize((256, 256))
    img = np.array(img)
    img = img / 255.0
    img = np.expand_dims(img, axis=0)
    
    # Make a prediction
    prediction = model.predict(img)
    label = np.argmax(prediction, axis=1)[0]
    
    # Return the predicted label
    if label == 0:
        return {"label": "Bed"}
    elif label == 1:
        return {"label": "Chair"}
    else:
        return {"label": "Sofa"}

